# Mixture-of-Agents in 46 lines of code

Reference to the Mixture of Experts (MoE) paper - https://github.com/togethercomputer/MoA


<img src="https://github.com/togethercomputer/MoA/raw/main/assets/moa-explained.png" height=400 width=600>

In [1]:
%pip install langchain-openai langchain-anthropic langchain-mistralai langchain getpass nest_asyncio

ERROR: Could not find a version that satisfies the requirement getpass (from versions: none)
ERROR: No matching distribution found for getpass
Note: you may need to restart the kernel to use updated packages.


In [5]:
import asyncio
import os
import getpass
from langchain_openai.chat_models import ChatOpenAI
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage

In [6]:
from nest_asyncio import apply
apply()

In [7]:
# Set the API keys for the models
if os.environ.get('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = getpass.getpass("Enter your OpenAI API key: ")
if os.environ.get('ANTHROPIC_API_KEY') is None:
    os.environ['ANTHROPIC_API_KEY'] = getpass.getpass("Enter your Anthropic API key: ")
if os.environ.get('MISTRAL_API_KEY') is None:
    os.environ['MISTRAL_API_KEY'] = getpass.getpass("Enter your Mistral API key: ")

In [8]:
user_prompt = "What are 3 fun things to do in SF?"
reference_models = [
    ChatOpenAI(model='gpt-4o'),
    ChatAnthropic(model="claude-3-5-sonnet-20240620"),
    ChatMistralAI(),
]
aggregator_model = ChatOpenAI(model='gpt-4o')
aggreagator_system_prompt = """You have been provided with a set of responses from various open-source models to the latest user query. Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

Responses from models:"""

async def run_llm(model):
    """Run a single LLM call with a reference model."""
    for sleep_time in [1, 2, 4]:
        try:
            response = await model.ainvoke([HumanMessage(content=user_prompt)])
            return response.content
        except Exception as e:
            print(f"Error with {model.__class__.__name__}: {e}")
            await asyncio.sleep(sleep_time)
    return "Failed to get response"

In [9]:
async def main():
    results = await asyncio.gather(*[run_llm(model) for model in reference_models])

    final_response = await aggregator_model.ainvoke(
        [
            SystemMessage(
                content=aggreagator_system_prompt
                + "\n"
                + "\n".join(
                    [f"{i+1}. {str(element)}" for i, element in enumerate(results)]
                )
            ),
            HumanMessage(content=user_prompt),
        ]
    )

    print(final_response.content)

asyncio.run(main())

San Francisco is a city full of diverse attractions and activities that cater to a variety of interests. Here are three fun things to do that will give you a great taste of what the city has to offer:

1. **Visit Alcatraz Island**:
   - **Why**: Alcatraz Island, once a notorious federal prison, is one of San Francisco's most iconic landmarks. The island offers a fascinating audio tour that provides chilling insights into the lives of the prisoners and the history of the facility.
   - **Highlights**: The ferry ride to the island provides stunning views of the San Francisco skyline and the Golden Gate Bridge. The tour itself is rich with historical anecdotes and engaging stories.

2. **Explore Golden Gate Park**:
   - **Why**: Spanning over 1,000 acres, Golden Gate Park is a massive urban oasis with a plethora of attractions and activities.
   - **Highlights**: Key spots include the California Academy of Sciences, the de Young Museum, the Japanese Tea Garden, the San Francisco Botanical